In [1]:
import numpy as np
import tensorflow as tf 
from keras.models import load_model

from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten,Subtract,Reshape
from keras.preprocessing import image
from keras.models import Model
from keras.layers import Dense, GlobalAveragePooling2D,Conv2D,MaxPooling2D,Input,Lambda,GlobalMaxPooling2D
from keras.regularizers import l2
from keras import backend as K
from keras.applications.vgg16 import VGG16
from skimage.io import imsave

from matplotlib.pyplot import imread
from skimage.transform import rescale, resize
import os
#1024x768
#300x960

In [2]:
# Data 
reshape_size = [224,224,3]
dataset_path = '../../datasets/VisualPhish/'
data_path = dataset_path + 'dataset_new_phishing_1366_768_phish/'
imgs_num = 1095#955

# Saved models and computed embeddings
margin = 2.2
#output_dir = './'
output_dir = '../train/'
model_name = '../train/model.h5'

phish_emb_name = 'phishing_emb.npy'
phish_emb_labels_name = 'phishing_labels.npy'

phish_train_idx_name = 'train_idx.npy'
phish_test_idx_name = 'test_idx.npy'

train_emb_name = 'whitelist_emb.npy'
train_emb_labels_name = 'whitelist_labels.npy'

# Load New phishing examples

In [3]:
def read_new_phish_imgs(data_path,targets,imgs_num,reshape_size,labels_dict):
    all_imgs = np.zeros(shape=[imgs_num,224,224,3])
    all_labels = np.zeros(shape=[imgs_num,1])
    all_file_names = []
    label_names = sorted(os.listdir(data_path))
    count = 0
    #print(targets)
    for i in range(0,len(targets)):
        #for j in range(0,imgs_num):
        label=targets[i]
        target_path = data_path + targets[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            if not file_names[j] == 'labels.txt':
                print(file_names[j])
                img = imread(os.path.join(target_path,file_names[j]))
                img = img[:,:,0:3]
                all_imgs[count,:,:,:] = resize(img, (reshape_size[0], reshape_size[1]),anti_aliasing=True)
                #print(label)
                #print(labels_dict)
                all_labels[count,:] = labels_dict[label]
                all_file_names.append(file_names[j])
                count = count + 1
    print(all_imgs.shape)
    return all_imgs,all_labels,all_file_names

#get names of labels/indices
def get_dict_targets(data_path,file_name):
    targets_file = open(data_path+file_name, "r")
    targets = targets_file.read()
    targets_list = targets.splitlines()
    counter=0
    x = {'absa':0}
    for i in range(0,len(targets_list)):
        label=targets_list[i]
        target_path = data_path + targets_list[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            if not( file_names[j] == 'textfiles' or file_names[j] == 'textfiles_translated' ): 
                first_file = file_names[j]
                first_file = file_names[j].replace('T','')
                #label = int(first_file.split('_')[0])
                #d = {targets_list[i]:label}
                #x.update(d)
                if(label not in x):
                    counter+=1
                    x[label]=counter
                break
    return x
        
labels_dict = get_dict_targets(dataset_path+'dataset_new_phishing_1366_768_phish/','targets.txt')
print(labels_dict)

{'absa': 0, 'adobe.com': 1, 'github.com': 2, 'player.vimeo.com': 3, 'wordpress.com': 4, 'amazon.com': 5, 'google.com': 6, 'play.google.com': 7, 'wordpress.org': 8, 'apache.org': 9, 'gravatar.com': 10, 'yahoo.com': 11, 'apple.com': 12, 'itunes.apple.com': 13, 'support.google.com': 14, 'youtube.com': 15, 'blogspot.com': 16, 'linkedin.com': 17, 'tumblr.com': 18, 'en.wikipedia.org': 19, 'microsoft.com': 20, 'twitter.com': 21, 'flickr.com': 22, 'nytimes.com': 23, 'wikipedia.org': 24}


In [4]:
targets_file = open(data_path+'labels.txt', "r")
targets = targets_file.read()
targets = targets.splitlines()
all_imgs_phish_new,all_labels_phish_new,all_file_names_phish_new = read_new_phish_imgs(data_path,targets,imgs_num,reshape_size,labels_dict)

062b64c2c0.png
0d7134f511.png
12da301d03.png
142a9f9316.png
154bc0b622.png
15fcfbc9bf.png
164987d787.png
16d8b1bea6.png
28a98eb5bb.png
2bde33ca87.png
37b6e83bc4.png
3c222291f1.png
3ee192c767.png
3f973c559a.png
4420b97707.png
4a462bd60e.png
4b6876362f.png
51b7588062.png
5210d51124.png
5748198a2b.png
59e54284df.png
616259120d.png
61bf841c3f.png
65392081ac.png
6690a5ef39.png
7469900ba8.png
7be9371c88.png
7e0fc57b7f.png
8fd218a07b.png
a728524d48.png
ab2c3deab0.png
b457de7d15.png
b959746dfd.png
ba24dffba4.png
bdb46878b6.png
be521c8c4a.png
c0b0c9524f.png
c46dd03fba.png
c48c8c1c3c.png
cce0cae153.png
d852585ec3.png
df419386ac.png
e03a29fa6b.png
e2333caffc.png
e5130433c0.png
ea37cc063f.png
ea4d50d915.png
ee30733838.png
ff89c4cecb.png
02b4ed442f.png
03241dcaa0.png
039473af4d.png
14b144fc15.png
2424e8d4a3.png
3097fca9b1.png
33de26de62.png
3b79b0e141.png
54f4b4b22f.png
6005615b5a.png
64133fb2f9.png
65777459e6.png
6f3ada220e.png
712f5a4603.png
776dc19871.png
7adf74ebcd.png
7eeb067f99.png
80ebcf49e7

0fbf625e32.png
1578dfdf0c.png
1e72ad8266.png
257be04434.png
2ac114cb84.png
3181ac5b3f.png
441bd2bf0d.png
460ca08ad1.png
476e20c045.png
49b66a794c.png
4c1d470718.png
4e8ceee157.png
62f3a321ca.png
6c32ebc6b5.png
754b3870a2.png
7a11a8f965.png
7dee59cec0.png
7fed322dd6.png
97f48f7ba8.png
a4658190fe.png
a745648b38.png
a95194556f.png
b08e8961da.png
b925bd7606.png
bf048f49df.png
c3b86ee966.png
c689e37801.png
c6daf0a808.png
ca3e4ac819.png
cbd5d508e6.png
cd5a263d4c.png
cfd1b95f68.png
da69ab1281.png
df97747b96.png
e6fb2437aa.png
f01856e3c2.png
f53d4f32e1.png
f806c734c3.png
fa50d1b854.png
fb23a380fb.png
fdbc41c3bc.png
fe9af407a2.png
09bbf37d06.png
09ed5e71a2.png
0abd76b300.png
0f801723fa.png
12b05ff969.png
12c8c0b0be.png
148ac69780.png
1675475765.png
19333f84f2.png
1949cbee12.png
1d45c8c356.png
1f214ce56a.png
210cd67f7f.png
342879437a.png
3de3611502.png
42c68d1627.png
43785727df.png
4adcab62c6.png
51b7a56a62.png
55a513a784.png
668b133e89.png
680e4978de.png
69a76b3697.png
6a12b3c8e7.png
6b901e7453

# Load model and predict

In [5]:
# Load_model 
def loss(y_true,y_pred):
    loss_value = K.maximum(y_true, margin + y_pred)
    loss_value = K.mean(loss_value,axis=0)
    return loss_value

final_model = load_model(model_name, custom_objects={'loss': loss})
inside_model = final_model.layers[3]

X_phish_new = inside_model.predict(all_imgs_phish_new)

# Load precomputed embeddings 

In [6]:
X_legit_train = np.load(output_dir+train_emb_name)
y_legit_train = np.load(output_dir+train_emb_labels_name)

X_phish = np.load(output_dir+phish_emb_name)
y_phish = np.load(output_dir+phish_emb_labels_name)

phish_test_idx = np.load(output_dir+phish_test_idx_name)
phish_train_idx = np.load(output_dir+phish_train_idx_name)

X_phish_test = X_phish[phish_test_idx,:]
y_phish_test = y_phish[phish_test_idx,:]

X_phish_train = X_phish[phish_train_idx,:]
y_phish_train = y_phish[phish_train_idx,:]

label_dict={}#reverse lookup for file name, return labels
# Get file names of each example 
def read_file_names(data_path,file_name):
    targets_file = open(data_path+file_name, "r")
    targets = targets_file.read()
    
    file_names_list = []
    targets_list = targets.splitlines()
    for i in range(0,len(targets_list)):
        target_path = data_path + targets_list[i]
        #new add stuff
        label=targets_list[i]
        file_names = sorted(os.listdir(target_path))
        for j in range(0,len(file_names)):
            file_names_list.append(file_names[j])
            #new add stuff
            label_dict[file_names[j]]=label
    return file_names_list

legit_file_names = read_file_names(dataset_path+'dataset_new_phishing_1366_768_trusted/','targets.txt')
phish_file_names = read_file_names(dataset_path+'dataset_new_phishing_1366_768_phish/','targets.txt')

phish_train_file_names = []
for i in range(0,phish_train_idx.shape[0]): 
    phish_train_file_names.append(phish_file_names[phish_train_idx[i]])
    
phish_test_file_names = []
for i in range(0,phish_test_idx.shape[0]):
    phish_test_file_names.append(phish_file_names[phish_test_idx[i]])

# def get_label_from_name(name):
#     first_half = name.split('_',1)[0]
#     number = int(first_half.replace('T',''))
#     return number
def get_label_from_name(name):
    label = label_dict[name]
    label_int = labels_dict[label]
    return label_int

# Compute distances

In [7]:
# L2 distance
def compute_distance_pair(layer1,layer2):
    diff = layer1 - layer2
    l2_diff = np.sum(diff**2) / X_phish_train.shape[1]
    return l2_diff

# Pairwise distance between query image and training
def compute_all_distances(test_matrix):
    train_size = X_phish_train.shape[0] + X_legit_train.shape[0]
    X_all_train = np.concatenate((X_phish_train,X_legit_train))
    pairwise_distance = np.zeros([test_matrix.shape[0],train_size])
    for i in range(0,test_matrix.shape[0]):
        pair1 = test_matrix[i,:]
        for j in range(0,train_size):
            pair2 = X_all_train[j,:]
            l2_diff = compute_distance_pair(pair1,pair2)
            pairwise_distance[i,j] = l2_diff
    return pairwise_distance

pairwise_distance = compute_all_distances(X_phish_new)

# Find Smallest n distances
def find_min_distances(distances,n):
    idx = distances.argsort()[:n]
    values = distances[idx]
    return idx,values

# Find names of examples with min distance
def find_names_min_distances(idx,values):
    names_min_distance = ''
    only_names = []
    distances = ''
    for i in range(0,idx.shape[0]):
        index_min_distance = idx[i]
        if (index_min_distance < X_phish_train.shape[0]):
            names_min_distance = names_min_distance + 'Phish: ' + phish_train_file_names[index_min_distance] +','
            only_names.append(phish_train_file_names[index_min_distance])   
        else:
            names_min_distance = names_min_distance + 'Legit: ' + legit_file_names[index_min_distance-X_phish_train.shape[0]] +','
            only_names.append(legit_file_names[index_min_distance-X_phish_train.shape[0]])   
        distances = distances + str(values[i]) + ','
    names_min_distance = names_min_distance[:-1]
    distances = distances[:-1]
    return names_min_distance,only_names,distances

# Find same-category website (matching is correct if it was matched to the same category (e.g. microsoft and outlook ))
parents_targets = ['microsoft','apple','google','alibaba']
sub_targets = [['ms_outlook','ms_office','ms_bing','ms_onedrive','ms_skype'],['itunes','icloud'],['google_drive'],['aliexpress']]

parents_targets_idx = [90,12,65,4]
sub_targets = [[150,152,151,149,148],[153,154],[147],[5]]

def check_if_same_category(img_label1,img_label2):
    if_same = 0
    if img_label1 in parents_targets_idx:
        if img_label2 in sub_targets[parents_targets_idx.index(img_label1)]:
            if_same = 1
    elif img_label1 in sub_targets[0]:
        if img_label2 in sub_targets[0] or img_label2 == parents_targets_idx[0]:
            if_same = 1
    elif img_label1 in sub_targets[1]:
        if img_label2 in sub_targets[1] or img_label2 == parents_targets_idx[1]:
            if_same = 1
    elif img_label1 in sub_targets[2]:
        if img_label2 in sub_targets[2] or img_label2 == parents_targets_idx[2]:
            if_same = 1
    return if_same

# Find if target is in the top closest n distances
def check_if_target_in_top(test_label,only_names):
    found = 0
    idx = 0
    #test_label = get_label_from_name(test_file_name)
    print('***')
    print('Target: '+str(test_label))
    for i in range(0,len(only_names)):
        label_distance = get_label_from_name(only_names[i])
        if int(label_distance) == int(test_label):# or check_if_same_category(test_label,label_distance) == 1:
            found = 1
            idx = i+1
            print('found')
            break
    return found,idx

#300x960

# Compute correct matches

In [8]:
n = 1 #Top-1 match
correct = 0

for i in range(0,X_phish_new.shape[0]):
    distances_to_train = pairwise_distance[i,:]
    idx,values = find_min_distances(np.ravel(distances_to_train),n)
    names_min_distance,only_names,min_distances = find_names_min_distances(idx,values)
    found,found_idx = check_if_target_in_top(all_labels_phish_new[i,0],only_names)
    print('Closest: '+names_min_distance)
    
    if found == 1:
        correct += 1

print("Correct match percentage: " + str(correct/X_phish_new.shape[0]))

***
Target: 1.0
found
Closest: Phish: 062b64c2c0.png
***
Target: 1.0
found
Closest: Legit: b959746dfd.png
***
Target: 1.0
found
Closest: Legit: 5a3852891e.png
***
Target: 1.0
found
Closest: Legit: 142a9f9316.png
***
Target: 1.0
found
Closest: Legit: c48c8c1c3c.png
***
Target: 1.0
found
Closest: Phish: 15fcfbc9bf.png
***
Target: 1.0
found
Closest: Phish: 164987d787.png
***
Target: 1.0
found
Closest: Legit: beb2dc461b.png
***
Target: 1.0
found
Closest: Legit: b7fac22979.png
***
Target: 1.0
found
Closest: Phish: 2bde33ca87.png
***
Target: 1.0
found
Closest: Phish: 062b64c2c0.png
***
Target: 1.0
found
Closest: Legit: 9df071ea4d.png
***
Target: 1.0
found
Closest: Legit: 333b010d54.png
***
Target: 1.0
found
Closest: Phish: 3f973c559a.png
***
Target: 1.0
found
Closest: Legit: a526aca5de.png
***
Target: 1.0
found
Closest: Legit: bcb4d7d76d.png
***
Target: 1.0
found
Closest: Legit: afcaf0b32c.png
***
Target: 1.0
found
Closest: Legit: 133b33a836.png
***
Target: 1.0
found
Closest: Phish: 5210d511

Closest: Legit: 504e102804.png
***
Target: 10.0
found
Closest: Legit: eab4d689c9.png
***
Target: 10.0
found
Closest: Legit: 58887fab82.png
***
Target: 10.0
found
Closest: Legit: 5ff4d17693.png
***
Target: 10.0
found
Closest: Phish: 63997f525e.png
***
Target: 10.0
found
Closest: Legit: 6734a65cff.png
***
Target: 10.0
found
Closest: Legit: 306494c675.png
***
Target: 10.0
found
Closest: Phish: 6f11ea1c0a.png
***
Target: 10.0
found
Closest: Legit: 401e08c3e3.png
***
Target: 10.0
found
Closest: Legit: 8dabe366f7.png
***
Target: 10.0
found
Closest: Legit: 9913bba436.png
***
Target: 10.0
found
Closest: Legit: 992390b0e9.png
***
Target: 10.0
found
Closest: Legit: 5e16d98c36.png
***
Target: 10.0
found
Closest: Phish: ad5e7b0403.png
***
Target: 10.0
found
Closest: Legit: b3d33f4d68.png
***
Target: 10.0
found
Closest: Phish: 63997f525e.png
***
Target: 10.0
found
Closest: Phish: 63997f525e.png
***
Target: 10.0
found
Closest: Legit: c212a8f0a2.png
***
Target: 10.0
found
Closest: Phish: c6fc3c3e64.p

Target: 19.0
found
Closest: Phish: 7b11d4a20a.png
***
Target: 19.0
found
Closest: Phish: 71cdafd11a.png
***
Target: 19.0
found
Closest: Legit: 8e76127873.png
***
Target: 19.0
found
Closest: Phish: 87a0c73c3a.png
***
Target: 19.0
found
Closest: Phish: 895634e574.png
***
Target: 19.0
found
Closest: Phish: 71cdafd11a.png
***
Target: 19.0
found
Closest: Legit: b1736a71f7.png
***
Target: 19.0
found
Closest: Phish: 87a0c73c3a.png
***
Target: 19.0
found
Closest: Phish: ac243d0bd7.png
***
Target: 19.0
found
Closest: Phish: 124612cbbf.png
***
Target: 19.0
found
Closest: Phish: ac243d0bd7.png
***
Target: 19.0
found
Closest: Phish: b1c6c0828e.png
***
Target: 19.0
Closest: Legit: 457a1b8d5c.png
***
Target: 19.0
Closest: Phish: ef09873741.png
***
Target: 19.0
found
Closest: Legit: ce80ffb19e.png
***
Target: 19.0
Closest: Legit: e18c4bae92.png
***
Target: 19.0
found
Closest: Legit: 8a31bf7958.png
***
Target: 19.0
found
Closest: Legit: 87770aff7c.png
***
Target: 19.0
Closest: Legit: 276032aeb3.png
**